# Adversarial Validation

# Test adversarial validation on a sample of data train set

In [1]:
from xgboost import XGBClassifier
import pandas as pd
import os
import sys
import numpy as np


PATH_PROJECT = 'C:\\Users\\alexa\\OneDrive\\Documents\\INSA\\5A\\Defi_IA\\defi_IA\\code\\'
PATH_UTILITIES = os.path.join(PATH_PROJECT, 'utilities')
sys.path.append(PATH_UTILITIES)
os.chdir(PATH_UTILITIES)


import data_loading as DL
import data_preparation_for_models as DP
import predictions_analysis as PA
from download_prediction import download_pred_Xtest

In [2]:
datatrain, Y, var_quant, var_quali, var_quali_to_encode = DL.main_load_data()

In [3]:
datatrain = datatrain.drop(['avatar_id'], axis=1)
datatrain.head()

,city,date,language,mobile,hotel_id,stock,group,brand,parking,pool,children_policy
0,paris,40,dutch,0,467,214,Accar Hotels,Safitel,0,0,0
1,paris,40,dutch,0,708,102,Yin Yang,Ardisson,1,0,0
2,paris,40,dutch,0,853,110,Chillton Worldwide,Tripletree,1,0,0
3,paris,40,dutch,0,207,108,Boss Western,J.Halliday Inn,0,0,1
4,paris,40,dutch,0,415,83,Boss Western,Boss Western,1,0,0


In [4]:
datatest = pd.read_csv("../../data/all_data/test_set_complet.csv")

In [5]:
datatest = datatest.drop(['order_requests', 'avatar_id'], axis=1)
datatest.head()

,city,date,language,mobile,hotel_id,stock,group,brand,parking,pool,children_policy
0,vilnius,21,romanian,0,161,46,Boss Western,J.Halliday Inn,1,0,0
1,vilnius,21,romanian,0,187,32,Accar Hotels,Marcure,1,1,0
2,vilnius,21,romanian,0,279,12,Independant,Independant,1,0,0
3,vilnius,21,romanian,0,395,10,Accar Hotels,Ibas,0,0,0
4,vilnius,21,romanian,0,488,42,Accar Hotels,Safitel,1,1,1


In [6]:
datatrain["is_test"] = 0
datatest["is_test"] = 1

In [7]:
datatrain = datatrain.sample(n=6640)
datatrain.head()

,city,date,language,mobile,hotel_id,stock,group,brand,parking,pool,children_policy,is_test
354929,madrid,17,finnish,1,97,37,Accar Hotels,Safitel,1,1,0,0
170575,rome,2,slovene,0,694,1,Independant,Independant,0,0,0,0
805456,paris,13,german,1,841,0,Independant,Independant,0,0,0,0
803945,paris,27,swedish,0,21,21,Accar Hotels,Ibas,0,0,0,0
883005,vienna,7,slovene,1,652,5,Boss Western,Boss Western,1,0,0,0


In [8]:
datatest.head()

,city,date,language,mobile,hotel_id,stock,group,brand,parking,pool,children_policy,is_test
0,vilnius,21,romanian,0,161,46,Boss Western,J.Halliday Inn,1,0,0,1
1,vilnius,21,romanian,0,187,32,Accar Hotels,Marcure,1,1,0,1
2,vilnius,21,romanian,0,279,12,Independant,Independant,1,0,0,1
3,vilnius,21,romanian,0,395,10,Accar Hotels,Ibas,0,0,0,1
4,vilnius,21,romanian,0,488,42,Accar Hotels,Safitel,1,1,1,1


In [9]:
frames = [datatrain, datatest]
data = pd.concat(frames,axis=0)

In [10]:
data_X = data.drop(["is_test"], axis=1)
data_Y = pd.DataFrame(data.is_test)

In [11]:
data_X = data_X.drop(["city", "language", "hotel_id", "group", "brand"], axis=1)

In [12]:
data_Y

,is_test
354929,0
170575,0
805456,0
803945,0
883005,0
...,...
6639,1
6640,1
6641,1
6642,1


## Dummy variables

In [13]:
# data_X = pd.get_dummies(data_X, columns=["city", "language", "hotel_id", "group", "brand"])

In [14]:
data_X.head()

,date,mobile,stock,parking,pool,children_policy
354929,17,1,37,1,1,0
170575,2,0,1,0,0,0
805456,13,1,0,0,0,0
803945,27,0,21,0,0,0
883005,7,1,5,1,0,0


In [15]:
# fit model no training data
model = XGBClassifier()
model.fit(data_X, data_Y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [16]:
# make predictions
y_pred = model.predict(data_X)
predictions = [round(value) for value in y_pred]

In [17]:
# evaluate predictions
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(data_Y, predictions)
accuracy = auc(fpr, tpr)
print("AUC: %.2f%%" % (accuracy * 100.0))

AUC: 85.12%


In [26]:
pd.DataFrame(y_pred)

,0
0,0
1,1
2,0
3,0
4,0
...,...
13279,1
13280,1
13281,1
13282,1


In [30]:
predVSreal = pd.DataFrame({'predictions': y_pred,
                          'real': data_Y.to_numpy().reshape(-1)})


In [47]:
# indexes of false discovery
idxs_of_fd = predVSreal.index[(predVSreal.predictions==1) & (predVSreal.real==0)]

# Script to select train data acccording to adversarial validation

In [72]:
# init dataset and batches
datatrain, Y, var_quant, var_quali, var_quali_to_encode = DL.main_load_data()
datatrain = datatrain.drop(['avatar_id'], axis=1)
datatrain["is_test"] = 0

datatrain_batch = np.array_split(datatrain, 100)
fd_dataframe = pd.DataFrame([])
estimated_accuracy = 0


# to iterate
for i in range(100):
    # get batch and fit classifier model
    train_sample = datatrain_batch[i]
    frames = [train_sample, datatest]
    data = pd.concat(frames,axis=0)
    data_X = data.drop(["is_test"], axis=1)
    data_Y = pd.DataFrame(data.is_test)
    data_X = data_X.drop(["city", "language", "hotel_id", "group", "brand"], axis=1)
    model = XGBClassifier()
    model.fit(data_X, data_Y)

    # make predictions
    y_pred = model.predict(data_X)
    predictions = [round(value) for value in y_pred]
    predVSreal = pd.DataFrame({'predictions': y_pred,
                              'real': data_Y.to_numpy().reshape(-1)})
    
    # update accuracy
    fpr, tpr, thresholds = roc_curve(data_Y, predictions)
    accuracy = auc(fpr, tpr)
    estimated_accuracy += accuracy
    # get False Discoveries (predicted test, were train)
    idxs_of_fd = predVSreal.index[(predVSreal.predictions==1) & (predVSreal.real==0)].to_list()
    fd_list = train_sample.iloc[idxs_of_fd,]
    fd_dataframe = pd.concat((fd_dataframe, fd_list), axis=0)
print("estimated accuracy of train/test classification (Monte Carlo): ", estimated_accuracy/100)

estimated accuracy of train/test classification (Monte Carlo):  0.8747720521710423


In [73]:
fd_dataframe

,city,date,language,mobile,hotel_id,stock,group,brand,parking,pool,children_policy,is_test
2,paris,40,dutch,0,853,110,Chillton Worldwide,Tripletree,1,0,0,0
9,paris,40,dutch,0,631,43,Yin Yang,8 Premium,1,0,0,0
14,paris,40,dutch,0,564,42,Accar Hotels,Ibas,0,0,0,0
17,paris,40,dutch,0,359,86,Accar Hotels,Navatel,0,0,0,0
23,paris,40,dutch,0,903,88,Independant,Independant,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
960667,vilnius,17,lithuanian,1,819,42,Yin Yang,Royal Lotus,0,0,0,0
960692,vilnius,17,lithuanian,1,498,9,Boss Western,Boss Western,0,0,0,0
960699,vilnius,17,lithuanian,1,47,23,Accar Hotels,Marcure,1,0,0,0
960702,vilnius,17,lithuanian,1,451,16,Independant,Independant,1,0,0,0


# Save selected data to csv

In [74]:
fd_dataframe.to_csv("../../data/adversarial_validation_data/selected_train_data.csv")

In [71]:
datatrain

,city,date,language,mobile,hotel_id,stock,group,brand,parking,pool,children_policy,is_test
0,paris,40,dutch,0,467,214,Accar Hotels,Safitel,0,0,0,0
1,paris,40,dutch,0,708,102,Yin Yang,Ardisson,1,0,0,0
2,paris,40,dutch,0,853,110,Chillton Worldwide,Tripletree,1,0,0,0
3,paris,40,dutch,0,207,108,Boss Western,J.Halliday Inn,0,0,1,0
4,paris,40,dutch,0,415,83,Boss Western,Boss Western,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
960779,sofia,7,slovene,1,141,0,Boss Western,J.Halliday Inn,1,0,0,0
960780,sofia,7,slovene,1,272,0,Yin Yang,8 Premium,0,0,0,0
960781,sofia,7,slovene,1,201,7,Chillton Worldwide,Tripletree,0,0,0,0
960782,sofia,7,slovene,1,542,0,Independant,Independant,0,0,0,0
